In [3]:

import torch
import torch.backends.cudnn as cudnn
cudnn.deterministic = True


import os
import os.path
import pickle

import sys
sys.path.insert(0, "D:/OneDrive/연구/ML/MyProject/AgentNet") #PC
#importlib.reload(ML)
import src.DataStructure as DS
from src.utils import *
from src.system import *
from src.model import *

%matplotlib inline
#%matplotlib notebook
torch.set_printoptions(precision = 10)

In [ ]:
class DataGen():
    def __init__(self, system):
        pass
    
    def run(self, file_name, total_size, batch_size, train_ratio):

        train_image = []
        train_label = []
        test_image = []
        test_label = []
        
        batch_num = int(total_size/batch_size)
        train_size = int(batch_num * train_ratio)
        test_size = int(batch_num*(1-train_ratio))
        
        for i in range(train_size):
            data, answer = next(system)
            train_image.append(data.astype(float))
            train_label.append(answer)

        train_output = {'Image':train_image, 'Label':train_label}
        
        for i in range(test_size):
            data, answer = next(system)
            test_image.append(data.astype(float))
            test_label.append(answer)
        
        test_output = {'Image':test_image, 'Label':test_label}
            
        # Output pickle
        with open(file_name + '_train.pkl', 'wb') as f:
            pickle.dump(train_output, f)
            
        with open(file_name + '_test.pkl', 'wb') as f:
            pickle.dump(test_output, f)

In [5]:
pp = {}
system = Flock_LSTM()

pp['ob_num'] = 'A'
pp['time_interval'] = 30
system.assign_pp(pp)

In [ ]:
def full_set(system, i):
    full_set = set([])
    for s in range(system.step_num+1):
        full_set = set.union(full_set, system.time_list[int(i+(s*system.time_interval))]['ID'])
    full_set = np.sort(np.array(list(full_set)))
    return full_set

In [ ]:
def lstm_next(self):

    while True:
        i = self.order[self.count]
        
        if self.count % 10 == 0:
            print(self.count)
        self.count += 1   
        full_set = set([])
        for s in range(system.step_num+1):
            full_set = set.union(full_set, self.time_list[int(i+(s*self.time_interval))]['ID'])
        full_set = np.sort(np.array(list(full_set)))
        
        data_input = self.time_list[i][['x', 'y', 'z', 'vx', 'vy', 'vz', 'hdg', 'hdg_rate', 'V']].values
        data_input = np.concatenate((data_input, np.ones((data_input.shape[0], 1))), axis=1)
        data = np.zeros((self.max_agent, self.state_num))
        data = np.concatenate((data, (-1)*np.ones((data.shape[0], 1))), axis=1)
        data[idx2idx(full_set, np.sort(np.array(list(set(self.time_list[int(i)]['ID'])))))] = data_input    
        
        answer_list = []

        previous_set = set([])
        present_set = set(self.time_list[int(i)]['ID'])
        for s in range(self.step_num):
            mask_input_new = np.zeros(self.max_agent)
            mask_input_next = np.zeros(self.max_agent)
            next_set = set(self.time_list[int(i + (s+1) * self.time_interval)]['ID'])
            
            # A : agents which exists in next step AND new entry (ready for hidden state init)
            new_set = present_set - previous_set
            mask_input_new[idx2idx(full_set, np.sort(np.array(list(new_set))))] = 1 
            
            # B1 : agents in next step
            mask_input_next[idx2idx(full_set, np.sort(np.array(list(next_set))))] = 1
            
            # B2 : agents which exists currently and next step (covers A)
            present_next_set = set.intersection(present_set, next_set)
            mask_input_next[idx2idx(full_set, np.sort(np.array(list(present_next_set))))] = 2 
            
            answer = np.zeros((self.max_agent, 2 + self.answer_num))
            answer[:, 0] = mask_input_new
            answer[:, 1] = mask_input_next

            for x in self.time_list[int(i + (s+1) * self.time_interval)]['ID']:
                idx_x = np.argwhere(full_set==x)[0][0]
                answer[idx_x][2:] = (
                self.time_list[int(i + (s+1) * self.time_interval)][self.time_list[int(i + (s+1) * self.time_interval)]['ID'] == x][
                    ['x', 'y', 'z', 'vx', 'vy', 'vz', 'hdg', 'hdg_rate', 'V']]).values

            answer_list.append(answer)
            previous_set = present_set
            present_set = next_set 

        return data, answer_list

def max_agent_next(system):
    max_agent = -1
    for i in range(0 , system.max_time, system.jump_interval):
        full_set = set([])
        for s in range(system.step_num+1):
            full_set = set.union(full_set, system.time_list[int(i+(s)*system.time_interval)]['ID'])
        max_agent = np.max((max_agent, len(full_set)))
    return max_agent

def max_agent_list(system):
    agent_list = []
    for i in range(0 , system.max_time, system.jump_interval):
        full_set = set([])
        for s in range(system.step_num+1):
            full_set = set.union(full_set, system.time_list[int(i+(s)*system.time_interval)]['ID'])
        agent_list.append(len(full_set))
    return agent_list

def collate_fn(batch):
    data, labels = zip(*batch)
    data, labels = torch.stack(data), torch.stack(labels)

    # data max
    
    data_checksum = torch.max(torch.argmax((data[:,:,-1]>0).double(), dim = 1)+1)
    seq_sum = torch.sum((labels[:,:,:,1]>1).float(), dim = 1)
    label_checksum = torch.max(torch.argmax((torch.sum(labels[:,:,:,1], dim = 1)>0).double(), dim = 1)+1)
    #print(data_checksum, label_checksum)
    cm = torch.max(data_checksum, label_checksum)
    seq_len = seq_sum[:, :cm] 

    data, labels = data[:,:cm,:], labels[:,:,:cm, :]

    mask_input_new = labels[:,:,:,0].unsqueeze(-1)
    mask_input_next = ((labels[:,:,:,1].unsqueeze(-1))>0).float()
    mask_input_check = ((labels[:,:,:,1].unsqueeze(-1))>1).float()
    labels = labels[:,:,:,2:]

    present = torch.zeros_like(mask_input_next[:,0,:,0])
    zero_present = torch.zeros_like(mask_input_next[:,0,:,0])
    target_mask_list = [[] for _ in range(labels.shape[1])]

    for i in range(labels.shape[1]):
        present += (mask_input_check[:,i,:,0]>0).float()
        for j in range(labels.shape[1]):
            target_mask_list[i].append((present==(j+1)).float())
        present = torch.where(present == seq_len, torch.zeros_like(zero_present), present)

    return data, labels, target_mask_list, mask_input_new, mask_input_next, mask_input_check

In [ ]:
# system loaded

system.step_num = 10
system.time_interval = 30
system.jump_interval = 20
system.max_time = len(system.time_list) - system.step_num * system.time_interval
system.order = np.arange(0, system.max_time, system.jump_interval)
system.count = 0
agent_list = max_agent_list(system)
system.max_agent = 1886 
with open('./data/Flock/system/'+'flock_lstm_OBA_agent_list.pkl', 'wb') as f:
    pickle.dump(agent_list,f)

In [ ]:
# Size parameters
single_data_size = 198
total_data_size = single_data_size * 1
batch_size = 1
train_ratio = 0.8
filename = ''

batch_num = int(total_data_size/batch_size)
train_size = int(batch_num * train_ratio)
test_size = int(batch_num*(1-train_ratio))

In [ ]:
# Generating new dataset
system.time_interval = 30
file_name = system.rule_name + '_OB'+str('A') + '_TI' + str(system.time_interval) + '_JI' + str(system.jump_interval) + '_LSTM'
generator = DataGen(system)
if not os.path.isfile(file_name+'_train.pkl'):
    generator.run(file_name, total_data_size, batch_size, train_ratio)
#generator.run(file_name, total_data_size, batch_size, train_ratio)